In [1]:
%load_ext autoreload
%autoreload 2
# Occupy a GPU for the model to be loaded 
%env CUDA_DEVICE_ORDER=PCI_BUS_ID
# GPU ID, if occupied change to an available GPU ID listed under !nvidia-smi
%env CUDA_VISIBLE_DEVICES=2

import numpy as np
from rdkit import Chem, DataStructs
import h5py, ast, pickle
import pandas as pd

from ddc_pub import ddc_v3 as ddc

env: CUDA_DEVICE_ORDER=PCI_BUS_ID
env: CUDA_VISIBLE_DEVICES=2


In [2]:
data = pd.read_csv('./datasets/OPD_Data/FP_TL_Training_Validation.csv')['smiles'].tolist()

In [3]:
binmols = [(Chem.MolFromSmiles(smiles)) for smiles in data]
mols_in = [Chem.rdchem.Mol.ToBinary(smiles) for smiles in binmols]

In [5]:
# All apriori known characters of the SMILES in the dataset
charset = 'a0bMAuGpgFL=(-2l+i)5mYtVnD\\%TPI8y1S3N7OZ/#CsRUfd@Be9Eo[WKcrH4]6h^$?'
# Apriori known max length of the SMILES in the dataset
maxlen = 445
# Name of the dataset
name = "opd_fp_complete"

dataset_info = {"charset": charset, "maxlen": maxlen, "name": name}

In [6]:
# Initialize a model
model = ddc.DDC(x              = mols_in,      # input
                y              = mols_in,      # output
                dataset_info = dataset_info,
                name = './models/opd_fp_complete'
               )

Initializing model in train mode.
Input type is 'binary mols'.
Model received 12254 train samples and 1362 validation samples.
Model: "model_3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
Encoder_Inputs (InputLayer)     [(None, 460, 67)]    0                                            
__________________________________________________________________________________________________
mol_to_latent_model (Model)     (None, 128)          733824      Encoder_Inputs[0][0]             
__________________________________________________________________________________________________
Decoder_Inputs (InputLayer)     [(None, 459, 67)]    0                                            
__________________________________________________________________________________________________
latent_to_states_model (Model)  [(None, 256), (None, 136192     

In [7]:
model.fit(epochs              = 100,         # number of epochs
          lr                  = 1e-3,        # initial learning rate for Adam, recommended
          model_name          = "opd_fp_tl", # base name to append the checkpoints with
          checkpoint_dir      = "./models/", # save checkpoints in the notebook's directory
          mini_epochs         = 10,          # number of sub-epochs within an epoch to trigger lr decay
          save_period         = 50,          # checkpoint frequency (in mini_epochs)
          lr_decay            = True,        # whether to use exponential lr decay or not
          sch_epoch_to_start  = 500,         # mini-epoch to start lr decay (bypassed if lr_decay=False)
          sch_lr_init         = 1e-3,        # initial lr, should be equal to lr (bypassed if lr_decay=False)
          sch_lr_final        = 1e-6,        # final lr before finishing training (bypassed if lr_decay=False)
          patience            = 25)          # patience for Keras' ReduceLROnPlateau (bypassed if lr_decay=True)


Model trained with dataset 200614_fp--1000--0.0282--0.0000010 that has maxlen=455 and charset=a0bMAuGpgFL=(-2l+i)5mYtVnD\%TPI8y1S3N7OZ/#CsRUfd@Be9Eo[WKcrH4]6h^$? for 100 epochs.
noise_std: 0.010000, lstm_dim: 256, dec_layers: 2, td_dense_dim: 0, batch_size: 256, codelayer_dim: 128, lr: 0.001000.

Epoch 00001: LearningRateScheduler reducing learning rate to 0.0010000000474974513.
Epoch 1/1000
5/4 - 11s - loss: 2.3586 - val_loss: 6.9807

Epoch 00002: LearningRateScheduler reducing learning rate to 0.0010000000474974513.
Epoch 2/1000
5/4 - 3s - loss: 0.9473 - val_loss: 7.8362

Epoch 00003: LearningRateScheduler reducing learning rate to 0.0010000000474974513.
Epoch 3/1000
5/4 - 3s - loss: 0.6501 - val_loss: 8.8744

Epoch 00004: LearningRateScheduler reducing learning rate to 0.0010000000474974513.
Epoch 4/1000
5/4 - 3s - loss: 0.5164 - val_loss: 9.4952

Epoch 00005: LearningRateScheduler reducing learning rate to 0.0010000000474974513.
Epoch 5/1000
5/4 - 3s - loss: 0.4361 - val_loss: 9.5

5/4 - 3s - loss: 0.1822 - val_loss: 0.4689

Epoch 00056: LearningRateScheduler reducing learning rate to 0.0010000000474974513.
Epoch 56/1000
5/4 - 3s - loss: 0.1792 - val_loss: 0.4925

Epoch 00057: LearningRateScheduler reducing learning rate to 0.0010000000474974513.
Epoch 57/1000
5/4 - 3s - loss: 0.1829 - val_loss: 0.4700

Epoch 00058: LearningRateScheduler reducing learning rate to 0.0010000000474974513.
Epoch 58/1000
5/4 - 3s - loss: 0.1767 - val_loss: 0.4890

Epoch 00059: LearningRateScheduler reducing learning rate to 0.0010000000474974513.
Epoch 59/1000
5/4 - 3s - loss: 0.1797 - val_loss: 0.4704

Epoch 00060: LearningRateScheduler reducing learning rate to 0.0010000000474974513.
Epoch 60/1000
5/4 - 3s - loss: 0.1777 - val_loss: 0.5122

Epoch 00061: LearningRateScheduler reducing learning rate to 0.0010000000474974513.
Epoch 61/1000
5/4 - 3s - loss: 0.1794 - val_loss: 0.4621

Epoch 00062: LearningRateScheduler reducing learning rate to 0.0010000000474974513.
Epoch 62/1000
5/4 - 


Epoch 00113: LearningRateScheduler reducing learning rate to 0.0010000000474974513.
Epoch 113/1000
5/4 - 3s - loss: 0.1466 - val_loss: 2.6340

Epoch 00114: LearningRateScheduler reducing learning rate to 0.0010000000474974513.
Epoch 114/1000
5/4 - 3s - loss: 0.1450 - val_loss: 24.4966

Epoch 00115: LearningRateScheduler reducing learning rate to 0.0010000000474974513.
Epoch 115/1000
5/4 - 3s - loss: 0.1399 - val_loss: 0.9554

Epoch 00116: LearningRateScheduler reducing learning rate to 0.0010000000474974513.
Epoch 116/1000
5/4 - 3s - loss: 0.1352 - val_loss: 0.6379

Epoch 00117: LearningRateScheduler reducing learning rate to 0.0010000000474974513.
Epoch 117/1000
5/4 - 3s - loss: 0.1347 - val_loss: 0.4178

Epoch 00118: LearningRateScheduler reducing learning rate to 0.0010000000474974513.
Epoch 118/1000
5/4 - 3s - loss: 0.1355 - val_loss: 0.3717

Epoch 00119: LearningRateScheduler reducing learning rate to 0.0010000000474974513.
Epoch 119/1000
5/4 - 3s - loss: 0.1303 - val_loss: 0.377


Epoch 00170: LearningRateScheduler reducing learning rate to 0.0010000000474974513.
Epoch 170/1000
5/4 - 3s - loss: 0.1073 - val_loss: 0.3035

Epoch 00171: LearningRateScheduler reducing learning rate to 0.0010000000474974513.
Epoch 171/1000
5/4 - 3s - loss: 0.1083 - val_loss: 0.3409

Epoch 00172: LearningRateScheduler reducing learning rate to 0.0010000000474974513.
Epoch 172/1000
5/4 - 3s - loss: 0.1072 - val_loss: 0.3542

Epoch 00173: LearningRateScheduler reducing learning rate to 0.0010000000474974513.
Epoch 173/1000
5/4 - 3s - loss: 0.1095 - val_loss: 0.3501

Epoch 00174: LearningRateScheduler reducing learning rate to 0.0010000000474974513.
Epoch 174/1000
5/4 - 3s - loss: 0.1091 - val_loss: 0.3595

Epoch 00175: LearningRateScheduler reducing learning rate to 0.0010000000474974513.
Epoch 175/1000
5/4 - 3s - loss: 0.1089 - val_loss: 0.3492

Epoch 00176: LearningRateScheduler reducing learning rate to 0.0010000000474974513.
Epoch 176/1000
5/4 - 3s - loss: 0.1122 - val_loss: 0.3377


Epoch 00227: LearningRateScheduler reducing learning rate to 0.0010000000474974513.
Epoch 227/1000
5/4 - 3s - loss: 0.0954 - val_loss: 0.3535

Epoch 00228: LearningRateScheduler reducing learning rate to 0.0010000000474974513.
Epoch 228/1000
5/4 - 3s - loss: 0.0954 - val_loss: 0.3342

Epoch 00229: LearningRateScheduler reducing learning rate to 0.0010000000474974513.
Epoch 229/1000
5/4 - 3s - loss: 0.0966 - val_loss: 0.2920

Epoch 00230: LearningRateScheduler reducing learning rate to 0.0010000000474974513.
Epoch 230/1000
5/4 - 3s - loss: 0.0984 - val_loss: 0.2863

Epoch 00231: LearningRateScheduler reducing learning rate to 0.0010000000474974513.
Epoch 231/1000
5/4 - 3s - loss: 0.0984 - val_loss: 0.2972

Epoch 00232: LearningRateScheduler reducing learning rate to 0.0010000000474974513.
Epoch 232/1000
5/4 - 3s - loss: 0.0982 - val_loss: 0.3067

Epoch 00233: LearningRateScheduler reducing learning rate to 0.0010000000474974513.
Epoch 233/1000
5/4 - 3s - loss: 0.0958 - val_loss: 0.3771

...

5/4 - 3s - loss: 0.0629 - val_loss: 0.1831

Epoch 00740: LearningRateScheduler reducing learning rate to 3.6570108807774935e-05.
Epoch 740/1000
5/4 - 3s - loss: 0.0604 - val_loss: 0.1761

Epoch 00741: LearningRateScheduler reducing learning rate to 3.60673495157403e-05.
Epoch 741/1000
5/4 - 3s - loss: 0.0619 - val_loss: 0.1723

Epoch 00742: LearningRateScheduler reducing learning rate to 3.557150206821388e-05.
Epoch 742/1000
5/4 - 3s - loss: 0.0622 - val_loss: 0.1750

Epoch 00743: LearningRateScheduler reducing learning rate to 3.508247144239793e-05.
Epoch 743/1000
5/4 - 3s - loss: 0.0628 - val_loss: 0.1733

Epoch 00744: LearningRateScheduler reducing learning rate to 3.460016392185111e-05.
Epoch 744/1000
5/4 - 3s - loss: 0.0642 - val_loss: 0.1736

Epoch 00745: LearningRateScheduler reducing learning rate to 3.4124487078528925e-05.
Epoch 745/1000
5/4 - 3s - loss: 0.0603 - val_loss: 0.1727

Epoch 00746: LearningRateScheduler reducing learning rate to 3.3655349755070956e-05.
Epoch 746/10

5/4 - 3s - loss: 0.0625 - val_loss: 0.1749

Epoch 00797: LearningRateScheduler reducing learning rate to 1.6612725220342935e-05.
Epoch 797/1000
5/4 - 3s - loss: 0.0633 - val_loss: 0.1726

Epoch 00798: LearningRateScheduler reducing learning rate to 1.6384336455779854e-05.
Epoch 798/1000
5/4 - 3s - loss: 0.0614 - val_loss: 0.1717

Epoch 00799: LearningRateScheduler reducing learning rate to 1.6159087538959187e-05.
Epoch 799/1000
5/4 - 4s - loss: 0.0606 - val_loss: 0.1716

Epoch 00800: LearningRateScheduler reducing learning rate to 1.5936935303817754e-05.
Epoch 800/1000
Model saved in re_train_fp_--800--0.1714--0.0000159.
5/4 - 3s - loss: 0.0628 - val_loss: 0.1714

Epoch 00801: LearningRateScheduler reducing learning rate to 1.571783717773164e-05.
Epoch 801/1000
5/4 - 3s - loss: 0.0614 - val_loss: 0.1746

Epoch 00802: LearningRateScheduler reducing learning rate to 1.5501751173357714e-05.
Epoch 802/1000
5/4 - 3s - loss: 0.0600 - val_loss: 0.1735

Epoch 00803: LearningRateScheduler reduc

5/4 - 3s - loss: 0.0622 - val_loss: 0.1743

Epoch 00854: LearningRateScheduler reducing learning rate to 7.546672630843898e-06.
Epoch 854/1000
5/4 - 3s - loss: 0.0615 - val_loss: 0.1748

Epoch 00855: LearningRateScheduler reducing learning rate to 7.442922330043763e-06.
Epoch 855/1000
5/4 - 3s - loss: 0.0631 - val_loss: 0.1719

Epoch 00856: LearningRateScheduler reducing learning rate to 7.340598369757217e-06.
Epoch 856/1000
5/4 - 3s - loss: 0.0619 - val_loss: 0.1712

Epoch 00857: LearningRateScheduler reducing learning rate to 7.2396811409108815e-06.
Epoch 857/1000
5/4 - 3s - loss: 0.0607 - val_loss: 0.1710

Epoch 00858: LearningRateScheduler reducing learning rate to 7.140151304013407e-06.
Epoch 858/1000
5/4 - 3s - loss: 0.0624 - val_loss: 0.1710

Epoch 00859: LearningRateScheduler reducing learning rate to 7.041989785449294e-06.
Epoch 859/1000
5/4 - 3s - loss: 0.0654 - val_loss: 0.1717

Epoch 00860: LearningRateScheduler reducing learning rate to 6.945177773823702e-06.
Epoch 860/100

5/4 - 3s - loss: 0.0619 - val_loss: 0.1727

Epoch 00911: LearningRateScheduler reducing learning rate to 3.4282314937339725e-06.
Epoch 911/1000
5/4 - 3s - loss: 0.0605 - val_loss: 0.1760

Epoch 00912: LearningRateScheduler reducing learning rate to 3.381100782480687e-06.
Epoch 912/1000
5/4 - 3s - loss: 0.0606 - val_loss: 0.1712

Epoch 00913: LearningRateScheduler reducing learning rate to 3.334618015786368e-06.
Epoch 913/1000
5/4 - 3s - loss: 0.0603 - val_loss: 0.1710

Epoch 00914: LearningRateScheduler reducing learning rate to 3.2887742858255128e-06.
Epoch 914/1000
5/4 - 3s - loss: 0.0620 - val_loss: 0.1706

Epoch 00915: LearningRateScheduler reducing learning rate to 3.243560807235814e-06.
Epoch 915/1000
5/4 - 3s - loss: 0.0600 - val_loss: 0.1710

Epoch 00916: LearningRateScheduler reducing learning rate to 3.1989689154345404e-06.
Epoch 916/1000
5/4 - 3s - loss: 0.0607 - val_loss: 0.1722

Epoch 00917: LearningRateScheduler reducing learning rate to 3.154990064958091e-06.
Epoch 917/1

5/4 - 3s - loss: 0.0632 - val_loss: 0.1739

Epoch 00968: LearningRateScheduler reducing learning rate to 1.557344772926136e-06.
Epoch 968/1000
5/4 - 3s - loss: 0.0586 - val_loss: 0.1726

Epoch 00969: LearningRateScheduler reducing learning rate to 1.5359346765108993e-06.
Epoch 969/1000
5/4 - 3s - loss: 0.0620 - val_loss: 0.1732

Epoch 00970: LearningRateScheduler reducing learning rate to 1.5148189222583469e-06.
Epoch 970/1000
5/4 - 3s - loss: 0.0591 - val_loss: 0.1718

Epoch 00971: LearningRateScheduler reducing learning rate to 1.4939934636052587e-06.
Epoch 971/1000
5/4 - 3s - loss: 0.0627 - val_loss: 0.1708

Epoch 00972: LearningRateScheduler reducing learning rate to 1.4734543096198368e-06.
Epoch 972/1000
5/4 - 3s - loss: 0.0618 - val_loss: 0.1715

Epoch 00973: LearningRateScheduler reducing learning rate to 1.4531975242368965e-06.
Epoch 973/1000
5/4 - 3s - loss: 0.0623 - val_loss: 0.1723

Epoch 00974: LearningRateScheduler reducing learning rate to 1.4332192255035751e-06.
Epoch 97